In [37]:
import pandas as pd
import psycopg2
import dotenv
import os
from socceraction.xthreat import ExpectedThreat
import matplotlib.pyplot as plt
import numpy as np

dotenv.load_dotenv()

PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DB")

conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DATABASE,
    user=PG_USER,
    password=PG_PASSWORD,
    port=PG_PORT,
    sslmode="require",
)

query = """
SELECT 
    s.id, s.game_id, s.period_id, s.seconds, 
    s.player_id, s.team_id, 
    s.start_x, s.start_y, s.end_x, s.end_y, 
    s.action_type, s.result, s.bodypart,
    p.player_name, t.team_name
FROM 
    spadl_actions s
LEFT JOIN 
    players p ON s.player_id = p.player_id
LEFT JOIN 
    teams t ON s.team_id = t.team_id
WHERE 
    s.start_x IS NOT NULL 
    AND s.end_x IS NOT NULL
	AND s.game_id = %s;
"""

# Load data into pandas DataFrame
df = pd.read_sql(query, conn, params=('5pcyhm34h5c948yji4oryevpw',))


# Preview the dataset
df.head()

C:\Users\ivanj\AppData\Local\Temp\ipykernel_28388\1258013567.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=('5pcyhm34h5c948yji4oryevpw',))


,id,game_id,period_id,seconds,player_id,team_id,start_x,start_y,end_x,end_y,action_type,result,bodypart,player_name,team_name
0,2068,5pcyhm34h5c948yji4oryevpw,1,0.0,6g9x1y7xmjzwhk646v1iyuup5,cyrrlv6l1onld5x247w1q1jlr,52.500,34.000,38.850,33.524,0,1,0,T. Verlinden,Beerschot
1,2069,5pcyhm34h5c948yji4oryevpw,1,1.0,cfxzvlgvt9jj17qxq41t9sxcl,cyrrlv6l1onld5x247w1q1jlr,41.055,32.640,33.915,26.452,0,1,0,R. Sanusi,Beerschot
2,2070,5pcyhm34h5c948yji4oryevpw,1,2.5,2ky2kn7gpjorkyg9zyg68pk6i,cyrrlv6l1onld5x247w1q1jlr,33.915,26.452,38.115,24.752,21,1,0,A. Konstantopoulos,Beerschot
3,2071,5pcyhm34h5c948yji4oryevpw,1,4.0,2ky2kn7gpjorkyg9zyg68pk6i,cyrrlv6l1onld5x247w1q1jlr,38.115,24.752,73.185,11.356,0,0,0,A. Konstantopoulos,Beerschot
4,2072,5pcyhm34h5c948yji4oryevpw,1,6.0,9fvwcsajeousbo5o0e84cj2sp,bw9wm8pqfzcchumhiwdt2w15c,82.635,12.716,48.720,2.176,0,0,1,H. Mendyl,OH Leuven


In [48]:
# Initialize the xT model
xT_model = ExpectedThreat()

# Define pitch dimensions
pitch_length = 105
pitch_width = 68

# Print the xT grid dimensions to understand the limits
print(f"xT grid dimensions: {xT_model.l} x {xT_model.w}")
print(f"Valid x indices: 0 to {xT_model.l - 1}")
print(f"Valid y indices: 0 to {xT_model.w - 1}")

# Process one row at a time to avoid vectorized operations that might cause index errors
def calculate_xt_values(df):
    # Normalize coordinates to [0, 1]
    df['start_x_norm'] = df['start_x'] / pitch_length
    df['start_y_norm'] = df['start_y'] / pitch_width
    df['end_x_norm'] = df['end_x'] / pitch_length
    df['end_y_norm'] = df['end_y'] / pitch_width
    
    # Create new columns for xT values
    df['start_xT'] = 0.0
    df['end_xT'] = 0.0
    
    # Process each row individually
    for idx, row in df.iterrows():
        try:
            # Scale normalized coordinates to grid dimensions
            start_x_idx = int(min(max(0, row['start_x_norm'] * xT_model.l), xT_model.l - 1))
            start_y_idx = int(min(max(0, row['start_y_norm'] * xT_model.w), xT_model.w - 1))
            end_x_idx = int(min(max(0, row['end_x_norm'] * xT_model.l), xT_model.l - 1))
            end_y_idx = int(min(max(0, row['end_y_norm'] * xT_model.w), xT_model.w - 1))
            
            # Store indices for debugging or visualization
            df.at[idx, 'start_x_idx'] = start_x_idx
            df.at[idx, 'start_y_idx'] = start_y_idx
            df.at[idx, 'end_x_idx'] = end_x_idx
            df.at[idx, 'end_y_idx'] = end_y_idx
            
            # Calculate xT values
            df.at[idx, 'start_xT'] = xT_model.xT[start_x_idx, start_y_idx]
            df.at[idx, 'end_xT'] = xT_model.xT[end_x_idx, end_y_idx]
        
        except Exception as e:
            print(f"Error at row {idx}:")
            print(f"  Original values: start_x={row['start_x']}, start_y={row['start_y']}, end_x={row['end_x']}, end_y={row['end_y']}")
            print(f"  Normalized: start_x_norm={row['start_x_norm']}, start_y_norm={row['start_y_norm']}")
            print(f"  Exception: {str(e)}")
            
            # Set default values for this row to avoid breaking the entire process
            df.at[idx, 'start_xT'] = 0.0
            df.at[idx, 'end_xT'] = 0.0
            df.at[idx, 'start_x_idx'] = 0
            df.at[idx, 'start_y_idx'] = 0
            df.at[idx, 'end_x_idx'] = 0
            df.at[idx, 'end_y_idx'] = 0
    
    # Compute xT contribution
    df['xT_contribution'] = (df['end_xT'] - df['start_xT']).round(5)
    
    return df

# Apply the function to your dataframe
df = calculate_xt_values(df)

xT grid dimensions: 16 x 12
Valid x indices: 0 to 15
Valid y indices: 0 to 11
Error at row 4:
  Original values: start_x=82.635, start_y=12.716000000000001, end_x=48.72, end_y=2.176000000000002
  Normalized: start_x_norm=0.787, start_y_norm=0.18700000000000003
  Exception: index 12 is out of bounds for axis 0 with size 12
Error at row 27:
  Original values: start_x=43.68000000000001, start_y=51.952, end_x=79.275, end_y=58.344
  Normalized: start_x_norm=0.4160000000000001, start_y_norm=0.764
  Exception: index 12 is out of bounds for axis 0 with size 12
Error at row 28:
  Original values: start_x=79.275, start_y=58.344, end_x=96.91499999999999, end_y=59.228
  Normalized: start_x_norm=0.755, start_y_norm=0.858
  Exception: index 12 is out of bounds for axis 0 with size 12
Error at row 29:
  Original values: start_x=96.91499999999999, start_y=59.228, end_x=101.42999999999999, end_y=60.316
  Normalized: start_x_norm=0.9229999999999999, start_y_norm=0.871
  Exception: index 14 is out of bou